In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)

In [2]:
df = pd.read_csv('../US_births.csv', low_memory=False)
df = df.sample(n=250_000, random_state=42)
df.shape

(250000, 55)

In [3]:
def drop_rows(df):
    df.drop(df[df['DBWT'].eq(9999)].index, inplace=True)
#     df['DBWT'] = np.where(df['DBWT'].eq(9999), df[df['DBWT'].lt(9999)]['DBWT'].mean(), df['DBWT'])
    
    df.drop(df[df['BMI'].eq(99.9)].index, inplace=True)
    df.drop(df[df['DBWT'].eq(9999)].index, inplace=True)
    df.drop(df[df['WTGAIN'].eq(99)].index, inplace=True)
    df.drop(df[df['PWgt_R'].eq(999)].index, inplace=True)
#     df.drop(df[df['FAGECOMB'].eq(99)].index, inplace=True)
    df.drop(df[df['DLMP_MM'].eq(99)].index, inplace=True)
    df.drop(df[df['DLMP_YY'].eq(9999)].index, inplace=True)
    return df


In [4]:
df = drop_rows(df)
df.shape

(231493, 55)

In [5]:
def feature_engineer(df):
    df['binary_sex'] = np.where(df['SEX'] == 'M', 1, 0)
    df['first_birth'] = np.where(df['ILLB_R'].eq(888), 1, 0)

    df['binary_sex'] = np.where(df['SEX'] == 'M', 1, 0)
    df['first_birth'] = np.where(df['ILLB_R'].eq(888), 1, 0)

    conditions = [df['CIG_0'].eq(0),
                  df['CIG_0'].eq(99),
    ]
    choices = ['None',
               'Unknown',
    ]
    df['smoked'] = np.select(conditions, choices, 'Daily')

    
    conditions = [df['PRIORDEAD'].eq(0),
                  df['PRIORDEAD'].eq(99),
    ]
    choices = ['None',
               'Unknown',
    ]
    df['PRIORDEAD_cat'] = np.select(conditions, choices, 'Yes')

    
    conditions = [df['PRIORTERM'].eq(0),
                  df['PRIORTERM'].eq(99),
    ]
    choices = ['None',
               'Unknown',
    ]
    df['PRIORTERM_cat'] = np.select(conditions, choices, 'Yes')

    
    conditions = [df['PRIORLIVE'].eq(0),
                  df['PRIORLIVE'].eq(99),
    ]
    choices = ['None',
               'Unknown',
    ]
    df['PRIORLIVE_cat'] = np.select(conditions, choices, 'Yes')

    
    conditions = [(df['DOB_MM'] > df['DLMP_MM']) & (2018 == df['DLMP_YY']),
                  (df['DOB_MM'] > df['DLMP_MM']) & (2018 > df['DLMP_YY']),
                  (df['DOB_MM'] < df['DLMP_MM']) & (2018 > df['DLMP_YY'])
    ]
    choices = [df['DOB_MM'] - df['DLMP_MM'],
               ((df['DOB_YY'] - df['DLMP_YY'])* 12) + df['DOB_MM'] - df['DLMP_MM'],
               ((df['DOB_YY'] - df['DLMP_YY'])* 12) - df['DLMP_MM'] + df['DOB_MM']

    ]
    df['pregnancy_length'] = np.select(conditions,choices, 12)

    conditions = [df['MAGER'].lt(18)
    ]
    choices = ['Minor'
    ]
    df['MAGER_cat'] = np.select(conditions,choices,'Adult')

    
    condition =[df['pregnancy_length'].eq(9),
                df['pregnancy_length'].eq(8),
                df['pregnancy_length'].eq(10),
                df['pregnancy_length'].lt(7)
    ]
    choices = ['9',
              '8',
               '10',
               'Early'
    ]
    df['pregnancy_length_cat'] = np.select(condition,choices, 'Late')

    
    df['BMI_log'] = np.log(df['BMI'])
    df['first_birth'] = np.where(df['ILLB_R'].eq(888), 1, 0)
    df['plural_delivery'] = np.where(df['ILLB_R'].lt(4), 'Yes', 'No')
    df['first_pregnancy'] = np.where(df['ILP_R'].eq(888), 1, 0)
    df['first_natal'] = np.where(df['ILOP_R'].eq(888),1, 0)
    df['PRECARE'] = np.where(df['PRECARE'].eq(99), 0, df['PRECARE'])
    df['PREVIS'] = np.where(df['PREVIS'].eq(99), 0, df['PREVIS'])
    df['35AGE_older'] = np.where(df['MAGER'].gt(34), 1, 0)

    df['pregnancy_length_WTGAIN'] = df['pregnancy_length']/df['PWgt_R']
    df['MOM_weight'] = (df['M_Ht_In']**2)*df['BMI']/704
    df['WTGAIN_div_MOM_weight'] = df['WTGAIN']/df['MOM_weight']
    df['WTGAIN_div_length'] = df['WTGAIN'] / df['pregnancy_length']
    df['Weight_before'] = df['MOM_weight'] - df['WTGAIN']
    df['WT_percent_gain'] = df['WTGAIN'] / df['PWgt_R']
    return df


In [6]:
df = feature_engineer(df)
df.shape

(231493, 75)

In [7]:
X = df[['ATTEND','BFACIL', 'smoked', 'DOB_MM', 'DMAR','FHISPX','FEDUC', 'FRACE6', 'first_birth', 'plural_delivery', 'first_pregnancy', 'first_natal',
        'IP_GON', 'LD_INDL', 'MAGER', '35AGE_older','MAR_IMP', 'MBSTATE_REC', 'MEDUC', 'MHISPX', 'MRAVE6', 'MTRAN', 'pregnancy_length', 'WTGAIN_div_MOM_weight','WTGAIN_div_length',
        'NO_INFEC','NO_MMORB','NO_RISKS','PAY', 'PAY_REC','PRECARE','PREVIS', 'PRIORDEAD_cat', 'PRIORLIVE_cat', 'PRIORTERM_cat', 'PWgt_R', 'BMI_log','M_Ht_In', 'MOM_weight',
        'RDMETH_REC', 'RESTATUS', 'RF_CESAR', 'SEX', 'WTGAIN','WT_percent_gain','MAGER_cat','pregnancy_length_cat','BMI'
]]
_X = pd.get_dummies(X, columns=['ATTEND', 'BFACIL','smoked', 'DOB_MM','DMAR','FHISPX','FEDUC','FRACE6', 'plural_delivery',
                                'IP_GON','LD_INDL', '35AGE_older','MAR_IMP', 'MBSTATE_REC', 'MEDUC', 'MHISPX', 'MRAVE6', 'MTRAN',
                                'NO_INFEC','NO_MMORB','NO_RISKS', 'PAY', 'PAY_REC','PRIORDEAD_cat', 'PRIORLIVE_cat','PRIORTERM_cat',
                                'RDMETH_REC', 'RESTATUS', 'RF_CESAR', 'SEX','MAGER_cat','pregnancy_length_cat'])
y = df['DBWT']

In [8]:
def feat_eng_dummy(_X):
    _X['MAGER_smoked_Daily'] = _X['MAGER'] * _X['smoked_Daily']
    _X['NO_RISKS_1_length'] = _X['NO_RISKS_1'] * _X['pregnancy_length']
    _X['RDMETH_REC_3_length'] = _X['RDMETH_REC_3'] * _X['pregnancy_length']
    _X['RDMETH_REC_1_length'] = _X['RDMETH_REC_1'] * _X['pregnancy_length']
    _X['ATTEND_1_length'] = _X['pregnancy_length'] * _X['ATTEND_1']
    _X['MRAVE6_1_FRACE6_1'] = _X['MRAVE6_1'] * _X['FRACE6_1']
    _X['BFACIL_1_length'] = _X['pregnancy_length'] * _X['BFACIL_1']
    _X['BMI_log_length'] = _X['BMI_log'] * _X['pregnancy_length']
    _X['M_Ht_In_length'] = _X['DMAR_1'] * _X['pregnancy_length']
    _X['LD_INDL_N_length'] = _X['LD_INDL_N'] * _X['pregnancy_length']
    _X['MTRAN_Y_length'] = _X['MTRAN_Y'] * _X['pregnancy_length']
    _X['PRECARE_length'] = _X['PRECARE'] * _X['pregnancy_length']
    _X['PREVIS_length'] = _X['PREVIS'] * _X['pregnancy_length']
    _X['MOM_weight_length'] = _X['MOM_weight'] * _X['pregnancy_length']

    _X['RDMETH_REC_3_pregnancy_length_cat_9'] = _X['RDMETH_REC_3'] * _X['pregnancy_length_cat_9']
    _X['RF_CESAR_Y_pregnancy_length_cat_9'] = _X['RF_CESAR_Y'] * _X['pregnancy_length_cat_9']
    return _X

In [9]:
_X = feat_eng_dummy(_X)

In [10]:

_X['DBWT'] = y
from statsmodels.formula.api import ols
formula = 'DBWT~BMI_log+WTGAIN+M_Ht_In+SEX_M+BFACIL_3+ATTEND_1+DMAR_1+FRACE6_1+FEDUC_3+FRACE6_1+first_birth+plural_delivery_Yes+LD_INDL_N\
+MBSTATE_REC_1+MEDUC_6+MHISPX_2+MRAVE6_2+MRAVE6_1+MTRAN_Y+NO_RISKS_1+PAY_1+PRECARE+PREVIS+PRIORDEAD_cat_Yes+RDMETH_REC_3+RF_CESAR_Y+MAGER_smoked_Daily+pregnancy_length+MOM_weight\
+WTGAIN_div_MOM_weight+WTGAIN_div_length+NO_RISKS_1_length+RDMETH_REC_3_length+RDMETH_REC_1_length+ATTEND_1_length+BFACIL_1_length+BMI_log_length+M_Ht_In_length\
+LD_INDL_N_length+LD_INDL_N_length+MTRAN_Y_length+NO_RISKS_1_length+PREVIS_length+MOM_weight_length+pregnancy_length_cat_9+pregnancy_length_cat_8\
+pregnancy_length_cat_10+pregnancy_length_cat_Early+RDMETH_REC_3_pregnancy_length_cat_9+RF_CESAR_Y_pregnancy_length_cat_9'
model = ols(formula=formula, data=_X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   DBWT   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     2831.
Date:                Thu, 30 Jul 2020   Prob (F-statistic):               0.00
Time:                        01:13:37   Log-Likelihood:            -1.7501e+06
No. Observations:              231493   AIC:                         3.500e+06
Df Residuals:                  231445   BIC:                         3.501e+06
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                           -5276.7282    329.974    -15.991      0.000   -5923.468   -4629.988
BMI_log                              1491.3511    120.360     12.391      0.000    1255.449    1727.253
WTGAIN                                 -4.3468      0.481     -9.030      0.000      -5.290      -3.403
M_Ht_In                                46.6253      1.018     45.813      0.000      44.631      48.620
SEX_M                                 115.5772      1.934     59.754      0.000     111.786     119.368
BFACIL_3                               88.0371     15.358      5.732      0.000      57.935     118.139
ATTEND_1                             -225.8802     28.986     -7.793      0.000    -282.691    -169.069
DMAR_1                                -85.4211     22.375     -3.818      0.000    -129.276     -41.566
FRACE6_1                               49.2113      2.679     18.367      0.000      43.960      54.463
FEDUC_3                               -10.0276      2.290     -4.380      0.000     -14.515      -5.540
first_birth                          -110.9862      2.223    -49.920      0.000    -115.344    -106.629
plural_delivery_Yes                  -695.3135      8.856    -78.515      0.000    -712.671    -677.956
LD_INDL_N                            -205.9150     27.043     -7.614      0.000    -258.919    -152.911
MBSTATE_REC_1                         -48.7461      2.500    -19.502      0.000     -53.645     -43.847
MEDUC_6                                14.7605      2.565      5.754      0.000       9.733      19.788
MHISPX_2                              -40.1979      7.123     -5.643      0.000     -54.159     -26.237
MRAVE6_2                             -111.8688      4.076    -27.448      0.000    -119.857    -103.881
MRAVE6_1                               29.5048      3.590      8.218      0.000      22.468      36.541
MTRAN_Y                              -997.4531     89.343    -11.164      0.000   -1172.563    -822.343
NO_RISKS_1                            268.7400     26.451     10.160      0.000     216.896     320.584
PAY_1                                 -23.5849      2.286    -10.318      0.000     -28.065     -19.105
PRECARE                                16.8971      0.670     25.231      0.000      15.584      18.210
PREVIS                                 32.1363      2.338     13.747      0.000      27.555      36.718
PRIORDEAD_cat_Yes                     -60.8634      9.286     -6.555      0.000     -79.063     -42.664
RDMETH_REC_3                        -1192.7425     27.551    -43.292      0.000   -1246.742   -1138.743
RF_CESAR_Y                           -146.6296     28.580     -5.131      0.000    -202.645     -90.614
MAGER_smoked_Daily                     -4.8329      0.132    -36.64